<center><img src="Slide1.png" width = "800"/></center>

### <b> InSAR Coherent Change Detection over Aleppo, Syria (2016)
    
<b><i> Corey Scher<sup>1</sup>
<i>Jamon Van Den Hoek<sup>2</sup>
    
<i><font size=2> 1. Department of Earth and Environmental Sciences | Graduate Center at the City University of New York
    
<i><font size=2> 2. College of Earth, Ocean, and Atmospheric Sciences | Oregon State University

### Introduction

   In this training we will walk through an example of bi-temporal coherent change detection over Aleppo, Syria to map proxies for urban damage that occurred as a result of air bombardments in 2016. We will be using the [Alaska Satellite Facility's](https://asf.alaska.edu/) [Vertex tool](https://search.asf.alaska.edu/#/) to generate two synthetic aperture radar [interferogram](https://appliedsciences.nasa.gov/sites/default/files/Session4-SAR-English_0.pdf) (InSAR) datasets to look for anomalous decreases in InSAR measurements of [coherence](https://en.wikipedia.org/wiki/Coherence_(physics) that occurred following a series of airstrike bombardments. This workflow can be completed either using the ASF [Hyp3 Python SDK](https://hyp3-docs.asf.alaska.edu/), as we will demonstrate in this notebook, or by generating InSAR coherence images manually using the ASF Vertex [user interface](https://search.asf.alaska.edu/#/) and raster band math in a geographic information system interface, such as [QGIS](https://www.qgis.org/en/site/).

In [189]:
import os
import folium
import pandas as pd
import asf_search as asf
from hyp3_sdk import HyP3
import shapely.geometry as shp
import matplotlib.pyplot as plt

In [71]:
#import credentials from .config file
credentials = pd.read_csv('/Users/coreyscher/Documents/GitHub/arset/hyp3.config', 
                          header = None, 
                          names = ['names', 'values']).set_index('names')

In [75]:
#authenticate to Hyp3 

hyp3 = HyP3(username=credentials.loc['username'][0], 
            password=credentials.loc['password'][0])

In [236]:
wkt = "POLYGON((37.0072 36.091,37.3326 36.091,37.3326 36.3183,37.0072 36.3183,37.0072 36.091))"
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1], 
                         intersectsWith=wkt, 
                         maxResults=10,
                        processingLevel = 'SLC',
                        end = '2016-09-30').data


In [110]:
def addFootprint(result):
    footprint = shp.Polygon(result.geometry['coordinates'][0])
    gjson = result.geojson()
    fid = gjson['properties']['fileID']
    return fid, footprint

In [237]:
gdf = gpd.GeoDataFrame(list(map(addFootprint, results)), 
                       columns = ['fileID', 'footprint'], 
                       geometry = 'footprint')

In [238]:
mapa = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')

gdf.iloc[0:3].explore(m = mapa, 
            column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [242]:
insar_stack  = asf.stack_from_id(gdf.iloc[1]['fileID'])

In [243]:
# Let's filter the stack for dates that are of interest

In [248]:
insar_stack[0].geojson()

{'type': 'Feature',
 'geometry': {'coordinates': [[[35.349876, 35.132954],
    [35.680817, 36.77219],
    [38.479492, 36.373482],
    [38.089169, 34.732533],
    [35.349876, 35.132954]]],
  'type': 'Polygon'},
 'properties': {'beamModeType': 'IW',
  'browse': [],
  'bytes': '8533024386',
  'centerLat': '35.7608',
  'centerLon': '36.9029',
  'faradayRotation': None,
  'fileID': 'S1A_IW_SLC__1SDV_20141007T033418_20141007T033445_002718_0030B9_BF2B-SLC',
  'fileName': 'S1A_IW_SLC__1SDV_20141007T033418_20141007T033445_002718_0030B9_BF2B.zip',
  'flightDirection': 'DESCENDING',
  'frameNumber': '473',
  'granuleType': 'SENTINEL_1A_FRAME',
  'groupID': 'S1A_IWDV_0473_0478_002718_021',
  'insarStackId': None,
  'md5sum': '98a2c0e9947918e58b7a9506775793b4',
  'offNadirAngle': None,
  'orbit': '2718',
  'pathNumber': '21',
  'perpendicularBaseline': -24,
  'platform': 'Sentinel-1A',
  'pointingAngle': None,
  'polarization': 'VV+VH',
  'processingDate': '2014-10-07T03:34:18.000000',
  'processin